In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt, matplotlib.image as mpimg
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
%matplotlib inline
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn import ensemble
from xgboost.sklearn import XGBClassifier
from sklearn.utils import shuffle
from datetime import date
import math
from sklearn.metrics import accuracy_score
from lightgbm import LGBMClassifier
from datetime import date
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc
from sklearn.metrics import roc_curve
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from catboost import CatBoostClassifier, Pool, cv

/home/alena/.local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Rosbank ML Competition
https://boosters.pro/champ_15
Прогнозирование оттока клиентов после льготного использования кредитной карты банка 

In [2]:
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')

In [3]:
train['sum'] = train['amount'].groupby(train['cl_id']).transform('sum')
test['sum'] = test['amount'].groupby(test['cl_id']).transform('sum')

In [14]:
test['sum'] = test['amount'].groupby(train['cl_id']).transform('sum')

In [4]:
train['MCC'].unique()

array([5200, 6011, 5921, 5411, 6012, 5814, 4111, 5812, 5661, 7996, 5947,
       5691, 5813, 7991, 7399, 4468, 5999, 5811, 5912, 5942, 9311, 5977,
       7832, 5499, 5651, 5944, 5621, 4112, 5735, 5995, 5331, 7230, 7922,
       4131, 4814, 8011, 5722, 4722, 5941, 4511, 5641, 7011, 9222, 5211,
       7299, 5993, 5533, 5655, 5983, 5992, 5541, 7531, 4829, 5251, 8999,
       5311, 5732, 8099, 5699, 5943, 4812, 5712, 7542, 8062, 5714, 5611,
       5462, 5451, 7997, 4011, 8071, 7512, 5261, 5719, 9399, 4784, 5631,
       7278, 7311, 9211, 7298, 6300, 4900, 5964, 5192, 5945, 5111, 5441,
       8043, 6051, 7538, 7032, 7523, 5399, 3011, 7216, 5946, 7221, 5300,
       8299, 4816, 4121, 8021, 6010, 4789, 7941, 5193, 5949, 7375, 7933,
       3010, 5950, 4215, 4214,  780, 7699, 5970, 5734, 5948, 7395, 5309,
       3703, 5994, 4411, 7999, 5511, 5131, 4899, 7392, 9402, 3381, 8398,
       5713, 5422, 5733,  742, 3355, 5199, 5310, 3260, 3058, 3628, 3824,
       3057, 3607, 3515, 3387, 7995, 7629, 7998, 62

In [7]:
train = pd.concat([train, pd.get_dummies(train['MCC'], prefix='mcc_')], axis=1)

In [9]:
train = pd.concat([train, pd.get_dummies(train['MCC'], prefix='mcc_')], axis=1)
test = pd.concat([test, pd.get_dummies(test['MCC'], prefix='mcc_')], axis=1)
train = pd.concat([train, pd.get_dummies(train['trx_category'], prefix='trx_category_')], axis=1)
test = pd.concat([test, pd.get_dummies(test['trx_category'], prefix='trx_category_')], axis=1)

In [19]:
test["target_flag"] = 0
test["target_sum"] = 15964680

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518376 entries, 0 to 518375
Data columns (total 9 columns):
PERIOD          518376 non-null object
cl_id           518376 non-null int64
MCC             518376 non-null int64
channel_type    516222 non-null object
currency        518376 non-null int64
TRDATETIME      518376 non-null object
amount          518376 non-null float64
trx_category    518376 non-null object
target_flag     518376 non-null int64
dtypes: float64(1), int64(4), object(4)
memory usage: 35.6+ MB


In [13]:
test['cl_id'].unique().shape

(5217,)

In [8]:
test.groupby('cl_id', as_index=False).first()[['cl_id', 'target_flag']].to_csv('subm.csv', encoding='utf-8', sep=',', index=False)

In [20]:
test.groupby('cl_id', as_index=False).first()[['cl_id', 'target_sum']].to_csv('subm_2.csv', encoding='utf-8', sep=',', index=False)

In [10]:
train['PERIOD'] = pd.to_datetime(train['PERIOD'])
train['month'] = train['PERIOD'].dt.day.astype(np.int32)
train['year'] = train['PERIOD'].dt.year.astype(np.int32)

In [11]:
test['PERIOD'] = pd.to_datetime(test['PERIOD'])
test['month'] = test['PERIOD'].dt.day.astype(np.int32)
test['year'] = test['PERIOD'].dt.year.astype(np.int32)

In [102]:
train['month'].unique()

array([10, 12,  7,  9,  8,  6,  4,  5,  3, 11,  1,  2])

In [14]:
train['MCC'] = train['MCC'].factorize()[0].astype(np.int32)
test['MCC'] = test['MCC'].factorize()[0].astype(np.int32)

train['currency'] = train['currency'].factorize()[0].astype(np.int32)
test['currency'] = test['currency'].factorize()[0].astype(np.int32)

train['trx_category'] = train['trx_category'].factorize()[0].astype(np.int32)
test['trx_category'] = test['trx_category'].factorize()[0].astype(np.int32)

train['channel_type'] = train['channel_type'].factorize()[0].astype(np.int32)
test['channel_type'] = test['channel_type'].factorize()[0].astype(np.int32)

In [12]:
train['year_month'] = train['year'].apply(str) + " " + train['month'].apply(str)
test['year_month'] = test['year'].apply(str) + " " + test['month'].apply(str)
train.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum,...,trx_category__CASH_ADV,trx_category__CAT,trx_category__DEPOSIT,trx_category__POS,trx_category__WD_ATM_OTHER,trx_category__WD_ATM_PARTNER,trx_category__WD_ATM_ROS,month,year,year_month
0,2017-01-10,0,5200,NaN,810,21OCT17:00:00:00,5023.0,POS,0,0.0,...,0,0,0,1,0,0,0,10,2017,2017 10
1,2017-01-10,0,6011,NaN,810,12OCT17:12:24:07,20000.0,DEPOSIT,0,0.0,...,0,0,1,0,0,0,0,10,2017,2017 10
2,2017-01-12,0,5921,NaN,810,05DEC17:00:00:00,767.0,POS,0,0.0,...,0,0,0,1,0,0,0,12,2017,2017 12
3,2017-01-10,0,5411,NaN,810,21OCT17:00:00:00,2031.0,POS,0,0.0,...,0,0,0,1,0,0,0,10,2017,2017 10
4,2017-01-10,0,6012,NaN,810,24OCT17:13:14:24,36562.0,C2C_OUT,0,0.0,...,0,0,0,0,0,0,0,10,2017,2017 10


In [105]:
train['year'] = train['year'].factorize()[0].astype(np.int32)
test['year'] = test['year'].factorize()[0].astype(np.int32)

train['month'] = train['month'].factorize()[0].astype(np.int32)
test['month'] = test['month'].factorize()[0].astype(np.int32)

In [106]:
train['c_year_month'] = train['year_month'].factorize()[0].astype(np.int32)
test['c_year_month'] = test['year_month'].factorize()[0].astype(np.int32)

In [13]:
train['tx']=train.groupby(['cl_id'])["amount"].transform("count").astype(np.int32)
test['tx']=test.groupby(['cl_id'])["amount"].transform("count").astype(np.int32)

In [15]:
train['n_MCC'] = train.groupby(['cl_id'])['MCC'].transform("nunique").astype(np.int32)
test['n_MCC'] = test.groupby(['cl_id'])['MCC'].transform("nunique").astype(np.int32)

In [16]:
train['n_trx_category'] = train.groupby(['cl_id'])['trx_category'].transform("nunique").astype(np.int32)
test['n_trx_category'] = test.groupby(['cl_id'])['trx_category'].transform("nunique").astype(np.int32)

In [17]:
train['n_month'] = train.groupby(['cl_id'])['month'].transform("nunique").astype(np.int32)
test['n_month'] = test.groupby(['cl_id'])['month'].transform("nunique").astype(np.int32)

In [18]:
train['sum_per_tx'] = train['sum'] / train['tx']
test['sum_per_tx'] = test['sum'] / test['tx']

In [ ]:
from datetime import datetime

def get_date(row):
    dt = datetime.strptime(row['TRDATETIME'], '%d%b%H:%M:%S:%f')
    return datetime(row['PERIOD'].year, dt.month, dt.day)


train['TRDATETIME_2'] = train.apply(lambda x: get_date(x), axis = 1)
train.head()

In [113]:
test['TRDATETIME_2'] = test.apply(lambda x: get_date(x), axis = 1)
test.head()

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,sum,month,year,year_month,c_year_month,tx,n_MCC,n_trx_category,n_month,sum_per_tx,TRDATETIME_2
0,2017-01-04,2,0,-1,0,18APR17:00:00:00,153.00,0,328717.72,0,0,2017 4,0,199,29,6,4,1651.847839,2017-04-18
1,2017-01-03,2,1,-1,0,10MAR17:00:00:00,388.08,0,328717.72,1,0,2017 3,1,199,29,6,4,1651.847839,2017-03-10
2,2017-01-02,2,2,-1,0,21FEB17:00:00:00,999.00,0,328717.72,2,0,2017 2,2,199,29,6,4,1651.847839,2017-02-21
3,2017-01-04,2,0,-1,0,26APR17:00:00:00,90.00,0,328717.72,0,0,2017 4,0,199,29,6,4,1651.847839,2017-04-26
4,2017-01-04,2,0,-1,0,08APR17:00:00:00,130.00,0,328717.72,0,0,2017 4,0,199,29,6,4,1651.847839,2017-04-08


In [29]:
train['amount_last'] = (train.groupby(['cl_id']).apply(lambda x: x.sort_values(["TRDATETIME_2"])))['amount'].transform('last')
test['amount_last'] = test.groupby(['cl_id']).apply(lambda x: x.sort_values(["TRDATETIME_2"]))['amount'].transform('last')

TypeError: last() missing 1 required positional argument: 'offset'

In [114]:
train['year_month'].unique()

array(['2017 10', '2017 12', '2017 7', '2017 9', '2017 8', '2017 6',
       '2017 4', '2017 5', '2017 3', '2016 11', '2016 12', '2017 1',
       '2017 2', '2017 11', '2018 1', '2018 2', '2018 3', '2018 4',
       '2016 10'], dtype=object)

In [115]:
train['c_year_month'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18])

In [23]:
train.loc[train.groupby('cl_id')['TRDATETIME_2'].idxmax()]['amount']

,PERIOD,cl_id,MCC,channel_type,currency,TRDATETIME,amount,trx_category,target_flag,target_sum,sum,month,year,tx,n_MCC,n_trx_category,n_month,sum_per_tx,TRDATETIME_2
2,2017-01-12,0,2,-1,0,05DEC17:00:00:00,767.00,0,0,0.00,64383.00,0,0,5,5,3,1,12876.600000,2017-12-05
32,2017-01-10,1,5,-1,0,19OCT17:00:00:00,342.00,0,0,0.00,267578.04,0,0,104,23,3,1,2572.865769,2017-10-19
109,2017-01-06,5,25,-1,0,18JUN17:00:00:00,3719.00,0,1,326211.75,546369.13,0,0,142,34,7,1,3847.669930,2017-06-18
268,2017-01-07,9,16,-1,0,31JUL17:15:51:03,100.00,0,0,0.00,849315.09,0,0,39,5,5,1,21777.310000,2017-07-31
370,2017-01-09,10,16,-1,0,28SEP17:05:14:47,50.00,0,0,0.00,1124343.99,0,0,463,36,7,1,2428.388747,2017-09-28
794,2017-01-09,11,7,-1,0,22SEP17:00:00:00,160.00,0,0,0.00,427747.14,0,0,217,39,2,1,1971.184977,2017-09-22
978,2017-01-08,14,5,-1,0,11AUG17:00:00:00,345.00,0,1,187776.76,448690.17,0,0,136,30,3,1,3299.192426,2017-08-11
1119,2017-01-05,20,3,-1,0,30MAY17:00:00:00,906.00,0,0,0.00,437308.09,0,0,77,17,2,1,5679.325844,2017-05-30
1315,2017-01-01,21,4,-1,0,28JAN17:00:00:00,1.00,7,0,0.00,338881.65,0,0,124,35,4,1,2732.916532,2017-01-28
1215,2017-01-07,22,79,-1,0,01JUL17:00:00:00,511.00,0,0,0.00,249767.00,0,0,59,12,6,1,4233.338983,2017-07-01


In [116]:
columns = ['cl_id',  'trx_category',  'amount', 'tx', 'n_MCC', 'n_trx_category', 'sum', 'c_year_month']

In [78]:
model = LGBMClassifier( n_jobs=-1)
model.fit(train[columns], train['target_flag'])

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [79]:
X_train, X_test, y_train, y_test = train_test_split(train[columns], train['target_flag'], test_size=0.33, random_state=42)
model = KNeighborsClassifier(n_neighbors=3)
model.fit(X_train, y_train)  
X_train['target_pred'] = model.predict(X_train)
X_train['target_pred'] = X_train['target_pred'].groupby(X_train['cl_id']).transform(lambda x:x.value_counts().index[0])
print(roc_auc_score(y_train, X_train['target_pred']))
X_test['target_pred'] = model.predict(X_test)
X_test['target_pred'] = X_test['target_pred'].groupby(X_test['cl_id']).transform(lambda x:x.value_counts().index[0])

print(roc_auc_score(y_test, X_test['target_pred']))
print(cross_val_score(model, y_test, X_test['target_pred'])) 
model.fit(train[columns], train['target_flag'])
train['target_pred'] = model.predict(train[columns])
train['target_pred'] = train['target_pred'].groupby(train['cl_id']).transform(lambda x:x.value_counts().index[0])
print(roc_auc_score(train['target_flag'], train['target_pred']))

0.9931476871861956
0.9365481862650121
0.9973141092507076


In [31]:
X_train['target_pred2'] = X_train['target_pred'].groupby(X_train['cl_id']).transform(lambda x:x.value_counts().index[0])

In [79]:
roc_auc_score(train['target_flag'], model.predict(train[columns]))

0.8878205289884776

In [117]:
new_train = train.groupby('cl_id', as_index=False).first()
new_test = test.groupby('cl_id', as_index=False).first()

In [34]:
new_train['ratio'] = new_train['last_amount'] / new_train['sum']

In [28]:
new_train['last_amount'] = train.loc[train.groupby('cl_id')['TRDATETIME_2'].idxmax()]['amount']
new_test['last_amount'] = test.loc[test.groupby('cl_id')['TRDATETIME_2'].idxmax()]['amount']

In [130]:
X_train, X_test, y_train, y_test = train_test_split(new_train[columns], new_train['target_flag'], test_size=0.33, random_state=42)
#model = LGBMClassifier( n_jobs=-1)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=15)

#model = CatBoostClassifier(
#    logging_level='Silent'
#)


model.fit(X_train, y_train)  #source.groupby(['Country','City']).agg(lambda x:x.value_counts().index[0])
X_train['target_pred'] = model.predict(X_train)
print(roc_auc_score(y_train, X_train['target_pred']))
X_test['target_pred'] = model.predict(X_test)
X_test['target_pred'] = X_test['target_pred'].groupby(X_test['cl_id']).transform(lambda x:x.value_counts().index[0])

print(roc_auc_score(y_test, X_test['target_pred']))
model.fit(new_train[columns], new_train['target_flag'])
new_train['target_pred'] = model.predict(new_train[columns])
print(roc_auc_score(new_train['target_flag'], new_train['target_pred']))

model.fit(new_train[columns], new_train['target_flag'])
new_test['target_flag'] = model.predict(new_test[columns])
new_test.groupby('cl_id', as_index=False).first()[['cl_id', 'target_flag']].to_csv('subm.csv', encoding='utf-8', sep=',', index=False)

0.9958949023130474
0.7149366655871028
0.9944218138702392


In [37]:
model.booster_.feature_importance()

array([619, 116, 557, 599, 353, 151, 587,  18])

In [54]:
def rmsle(real, predicted):
    real = np.array(real)
    predicted = np.array(predicted)
    return np.sqrt(1.0 / len(real) * sum((np.log(real+1) - np.log(predicted+1)) ** 2))

def scale(y):
    return  (y - x_min) / (x_max - x_min)

def inverse(y):
    return y * (x_max - x_min) + x_min
    
from sklearn.ensemble import GradientBoostingRegressor
    

X_train, X_test, y_train, y_test = train_test_split(new_train, new_train['target_sum'], test_size=0.33, random_state=42)

columns_2 = ['cl_id',  'trx_category',  'amount', 'tx', 'n_MCC', 'n_trx_category', 'sum', 'target_flag']


from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import BaggingRegressor 
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
model = DecisionTreeRegressor()
model.fit(X_train[columns_2], y_train)
X_train['target_sum_pred'] = model.predict(X_train[columns_2])
print(rmsle(y_train, X_train['target_sum_pred']))
X_test['target_sum_pred'] = model.predict(X_test[columns_2])
print(rmsle(y_test, X_test['target_sum_pred']))
model.fit(train[columns_2], train['target_sum'])
train['target_sum_pred'] = model.predict(train[columns_2])
print(rmsle(train['target_sum'], train['target_sum_pred']))

/home/alena/.local/lib/python3.6/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/alena/.local/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


0.0
2.0237775848203947
2.0284221587630262e-15


In [40]:
model.feature_importances_

array([2.24079277e-02, 2.78583372e-08, 5.05653695e-07, 5.74297705e-02,
       5.54727388e-01, 1.43617017e-02, 3.46910840e-01, 4.16183851e-03])